<a href="https://colab.research.google.com/github/Arevik2004/arevik/blob/main/%D0%BB%D0%B0%D0%B1%D0%B0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Отчет по лабораторной работе на тему:Redis
# Выполнила работу студент группы ПИН-Б-З-22-1
# Элбакян Аревик Ашотовна


# Установите драйвер вашего любимого языка программирования и подключитесь к серверу Redis. Вставьте и увеличьте значение в рамках транзакции. Используя выбранный вами драйвер, создайте программу, которая считывает блокирующий список и выводит данные куда-нибудь (консоль, файл, Socket.io и т.д.), а также другую программу,которая записывает в тот же список

In [ ]:
import redis
import time

# Подключение к Redis
r = redis.Redis(
    host='localhost',
    port=6379,
    decode_responses=True
)

def transaction_example():
    try:
        pipeline = r.pipeline()
        pipeline.multi()
        pipeline.set('my_counter', 0)

        pipeline.incr('my_counter')
        pipeline.incrby('my_counter', 5)
        pipeline.incr('my_counter')

        # Выполняем все команды как одну транзакцию
        results = pipeline.execute()

        print(f"Транзакция выполнена успешно!")
        print(f"Результаты: {results}")
        print(f"Финальное значение счетчика: {r.get('my_counter')}")

    except redis.exceptions.RedisError as e:
        print(f"Ошибка при выполнении транзакции: {e}")

if __name__ == "__main__":
    transaction_example()

In [ ]:
import redis
import json
import logging
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('redis_consumer.log'),
        logging.StreamHandler()
    ]
)

class RedisConsumer:
    def __init__(self, host='localhost', port=6379):
        self.redis_client = redis.Redis(
            host=host,
            port=port,
            decode_responses=True
        )
        self.list_key = 'blocking_queue'

    def consume_messages(self, timeout=0):

        logging.info(f"Запуск consumer. Ожидание сообщений из списка '{self.list_key}'...")

        try:
            while True:
                # Блокирующее чтение из списка
                result = self.redis_client.blpop(self.list_key, timeout=timeout)

                if result:
                    key, message = result
                    self.process_message(message)
                else:
                    logging.info("Таймаут ожидания сообщений")
                    break

        except KeyboardInterrupt:
            logging.info("Consumer остановлен пользователем")
        except Exception as e:
            logging.error(f"Ошибка в consumer: {e}")

    def process_message(self, message):
        """Обработка полученного сообщения"""
        try:
            # Пытаемся распарсить как JSON
            data = json.loads(message)
            output = f"Получено сообщение: {json.dumps(data, ensure_ascii=False)}"
        except:
            # Если не JSON, выводим как есть
            output = f"Получено сообщение: {message}"

        # Выводим в консоль
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {output}")

        # Записываем в файл
        with open('output_messages.txt', 'a', encoding='utf-8') as f:
            f.write(f"{datetime.now().isoformat()}: {message}\n")

        logging.info(f"Обработано сообщение: {message[:50]}...")

if __name__ == "__main__":
    consumer = RedisConsumer()
    consumer.consume_messages(timeout=0)  # Бесконечное ожидание

In [ ]:
import redis
import json
import time
import random
from datetime import datetime
import threading

class RedisProducer:
    def __init__(self, host='localhost', port=6379):
        self.redis_client = redis.Redis(
            host=host,
            port=port,
            decode_responses=True
        )
        self.list_key = 'blocking_queue'

    def send_message(self, message):
        try:
            # RPUSH добавляет элемент в конец списка
            result = self.redis_client.rpush(self.list_key, message)
            print(f"[{datetime.now().strftime('%H:%M:%S')}] Отправлено: {message}")
            return result
        except Exception as e:
            print(f"Ошибка при отправке: {e}")
            return None

    def send_json_message(self, data):
        message = json.dumps(data, ensure_ascii=False)
        return self.send_message(message)

    def interactive_mode(self):
        print("Интерактивный режим producer")
        print("Введите сообщения для отправки (или 'quit' для выхода):")

        message_id = 1
        while True:
            try:
                user_input = input("Сообщение: ").strip()

                if user_input.lower() in ['quit', 'exit', 'q']:
                    break

                if user_input:
                    # Создаем структурированное сообщение
                    message_data = {
                        'id': message_id,
                        'text': user_input,
                        'timestamp': datetime.now().isoformat(),
                        'source': 'interactive'
                    }

                    self.send_json_message(message_data)
                    message_id += 1

            except KeyboardInterrupt:
                print("\nProducer остановлен")
                break

    def auto_mode(self, interval=2):
        """Автоматическая отправка тестовых сообщений"""
        print(f"Автоматический режим. Интервал: {interval} сек.")

        message_id = 1
        try:
            while True:
                # Создаем тестовое сообщение
                message_data = {
                    'id': message_id,
                    'message': f'Тестовое сообщение #{message_id}',
                    'timestamp': datetime.now().isoformat(),
                    'value': random.randint(1, 100),
                    'source': 'auto_producer'
                }

                self.send_json_message(message_data)
                message_id += 1
                time.sleep(interval)

        except KeyboardInterrupt:
            print("\nАвтоматический режим остановлен")

if __name__ == "__main__":
    producer = RedisProducer()

    print("Выберите режим работы producer:")
    print("1. Интерактивный режим")
    print("2. Автоматический режим")

    choice = input("Ваш выбор (1 или 2): ").strip()

    if choice == '1':
        producer.interactive_mode()
    elif choice == '2':
        try:
            interval = float(input("Интервал отправки (сек): ").strip() or 2)
        except ValueError:
            interval = 2
        producer.auto_mode(interval)
    else:
        print("Неверный выбор")